In [1]:
import numpy as np
from scipy.stats import unitary_group
from numpy import linalg as LA
from scipy.linalg import block_diag
from scipy.linalg import logm, expm
import qiskit

In [2]:
#define functions

def h(xx):
    x=np.real(xx)
    return -x*np.log2(x)-(1-x)*np.log2(1-x)
def H(v):
    summ=0
    d=len(v)
    for j in range(d):
        summ=summ+h(v[j])
    return summ

#rotates gamma0=diag(w) with U, extract blocks wrt Y=(2,2,1), and returns block spectrum v
def vvector(w,U):
    gamma0=np.diag(w)
    Udag=np.conjugate(U.T)
    gamma=np.matmul(U,np.matmul(gamma0,Udag))
    block1=gamma[0:2,0:2]
    block2=gamma[2:4,2:4]
    block1ev=LA.eigvals(block1)
    block2ev=LA.eigvals(block2)
    v=np.array([block1ev[0].real,block1ev[1].real,block2ev[0].real,block2ev[1].real,gamma[4][4].real])
    return(v)

#returns a unitary epsilon-close to identity
def unitaryclosetoid(epsilon):
    M = np.random.rand(d, d)-0.5 + 1j*np.random.rand(d, d)  # a (very basic) random complex matrix
    A = (M - np.conj(M.T)) #make it antihermitian. eigenvalues are like between -3i and 3i...
    deltaU=expm(epsilon*A) #unitary close to identity
    return(deltaU)

#shrinks and improves a podium. call it as podium2=newpodium(w, podium1, ...)
def newpodium(w,oldpodium,mnewpodium,localm,eps): #newmpodium = length of new podium; localm and eps = parameters for local maximization
    podium=np.copy(oldpodium) #so as not to change oldpodium, just in case
    deltaU=np.identity(d,dtype=complex)
    U=np.identity(d,dtype=complex)
    mpodium=len(podium)
    v=np.zeros(d)
    I=0.
    for k in range(localm):
        deltaU=unitaryclosetoid(eps)
        for l in range(mpodium): #treat  podium couples  (I,U) independently
            U = np.matmul(podium[l][1],deltaU) #unitary close to podium[l][1]
            v=vvector(w,U)
            I=H(v)
            if I>podium[l][0]:
                podium[l][0]=I
                podium[l][1]=np.copy(U) 
    podium = sorted(podium, key=lambda x: x[0])
    newpodium=[podium[-mnewpodium+j] for j in range(mnewpodium)]
    return(newpodium)

In [3]:
#choose w vector or generate it randomly and get vconj = conjectured value of vbest

#w=np.array(np.array([0.07633751, 0.12426679, 0.15111039, 0.30446614, 0.34381918]))
#e.g. [0.5, 0.5, 0.5, 0.8, 0.8] is an interesting one...or those from Kaustav's list
w=np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])  
w=np.sort(w)
t=sum(w)
d=len(w)

regime=0
if t/5<(w[1]+w[2])/2:
    a=(w[1]+w[2])/2
    b=(w[0]+w[3]+w[4])/3
    vconj=np.sort(np.array([a,a,b,b,b]))
    Iconj=H(vconj)
    regime=1
    print("1st regime")
elif t/5>(w[2]+w[3])/2:
    a=(w[2]+w[3])/2
    b=(w[0]+w[1]+w[4])/3
    vconj=np.sort(np.array([a,a,b,b,b]))
    Iconj=H(vconj)
    print("2nd regime")
    regime=2
elif t/5<(w[0]+w[3])/2:
    a=(w[0]+w[3])/2
    b=(w[1]+w[2]+w[4])/3
    vconj=np.sort(np.array([a,a,b,b,b]))
    Iconj=H(vconj)
    print("3rd regime")
    regime=3
elif t/5>(w[1]+w[4])/2:
    a=(w[1]+w[4])/2
    b=(w[0]+w[2]+w[3])/3
    vconj=np.sort(np.array([a,a,b,b,b]))
    Iconj=H(vconj)
    print("4th regime")
    regime=4
else:
    print("5th (identity) regime?")
    vconj=np.array([t/d,t/d,t/d,t/d,t/d])
    Iconj=H(vconj)   
    regime=5

5th (identity) regime?


In [4]:
#set code parameters

#number of global unitaries
m=1000000 #still the main (?) bottleneck parameter.
#looks like 10 mln is enough to "fall close to global maximum". after that local maximization is efficient (?)

#number of best global unitaries to keep on the first 'podium'
mpodium1=500
#number of best unitaries to keep on the second 'podium'
mpodium2=20
#parameters for shortlisting first podium into second podium
mlocal1=100 #number of close-to-identity unitaries 
epsilon1=0.05 #how close to identity
#parameters for shortlisting second podium into winner
mlocal2=30000
epsilon2=0.0005 
#parameters to further optimize winner
mlocal3=100000
epsilon3=0.0001
mlocal4=100000
epsilon4=0.000001

#i tried to fine-tune the parameters above but probably their values can be improved. One might also
#consider adding a third podium, i.e. one more step in the  box below...


In [5]:
#run optimization

#allocate memory for some relevant variables
vbest=np.array([w[j] for j in range(d)])
v=np.zeros(d)
Ibest=0.
I=0.
Ubest=np.identity(d,dtype=complex)
U=np.identity(d,dtype=complex)
podium1=[[0.,np.identity(d,dtype=complex)] for i in range(mpodium1)] #will be filled with couples (I value, U)
podium2=[[0.,np.identity(d,dtype=complex)] for i in range(mpodium2)]

#step 0: global optimization
for k in range(m):
    U=unitary_group.rvs(d)
    v=vvector(w,U)
    I=H(v)
    if I>podium1[0][0]:
        podium1[0][0]=I
        podium1[0][1]=np.copy(U)
        #order podium1; best (I,U) is given by podium1[-1]
        podium1 = sorted(podium1, key=lambda x: x[0])
                               
Ibest1=podium1[-1][0]

#step 1: shrink podium1 to podium2

podium2=np.copy(newpodium(w,podium1,mpodium2,mlocal1,epsilon1))
Ibest2=podium2[-1][0]

#the following count might have the following meaning: if the very first (2-5) unitaries on the podium2
#are close to each other, it's a hint indicating that we are hopefully sampling enough. Instead, if the 
#winning unitary is far from the second unitary, this might indicate that we are not sampling enough
#and that even the winning unitary might be stuck in a local minimum

count=0
for j in range(mpodium2):
    if Ibest2-podium2[j][0]<0.0001:
        count=count+1
print("Out of ", mpodium2, " couples (I,U) on the second podium, the first ",count, " couples are close to the first on 2nd podium")
 
#step 2: shrink podium2 to podium3=winner

winner=np.copy(newpodium(w,podium2,1,mlocal2,epsilon2))
Ibest3=winner[0][0]


#step 3: further local optimization
#assuming the winner Ubest is actually good, one can push the local maximization even further to get more digits
#we are shrinking podium 2 to a 1-position podium -> can still use function newpodium

winner=np.copy(newpodium(w,winner,1,mlocal3,epsilon3))
winner=np.copy(newpodium(w,winner,1,mlocal4,epsilon4))
#here even more rounds sometimes help! 
#winner=np.copy(newpodium(w,winner,1,300000,0.000001))
#winner=np.copy(newpodium(w,winner,1,300000,0.0000001))


Ibest=winner[0][0]
Ubest=np.copy(winner[0][1])

/software/opt/focal/x86_64/python/3.10-2022.08/lib/python3.10/site-packages/numpy/lib/function_base.py:935: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


Out of  20  couples (I,U) on the second podium, the first  1  couples are close to the first on 2nd podium


In [6]:
#print results

print("estimated Imax value after step 0 = global minimization: ",Ibest1)
print("estimated Imax value after step 1 = local minimization giving podium1 -> podium2: ",Ibest2)
print("estimated Imax value after step 2 = local minimization giving podium 2 -> winner: ",Ibest3)
print("estimated Imax value after step 3 = local further minimization of winner: ",Ibest)
print("upper bound: ", d*h(t/d))

estimated Imax value after step 0 = global minimization:  4.921541220284641
estimated Imax value after step 1 = local minimization giving podium1 -> podium2:  4.936197833926501
estimated Imax value after step 2 = local minimization giving podium 2 -> winner:  4.9405103346907255
estimated Imax value after step 3 = local further minimization of winner:  4.940510372207425
upper bound:  4.940510372207472


In [7]:
#unitarity check of Ubest

#might be relevant because in local optimization we are multiplying unitaries together and errors might add up (not the case so far, luckily)
Ubestdag=np.conjugate(Ubest.T)
np.matmul(Ubest,Ubestdag).round(14)

array([[ 1.+0.j, -0.+0.j, -0.+0.j,  0.-0.j, -0.-0.j],
       [-0.-0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [-0.-0.j,  0.-0.j,  1.+0.j,  0.-0.j, -0.+0.j],
       [ 0.+0.j,  0.-0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [-0.+0.j,  0.-0.j, -0.-0.j,  0.-0.j,  1.+0.j]])

In [8]:
#check conjecture

vbest=np.sort(vvector(w,Ubest))
print("vbest: ",vbest.round(6))
print("vconj: ",vconj.round(6))

print("\nEstimated Imax value: ",Ibest)
print("Conjectured Imax value: ",Iconj)
diff=Iconj-Ibest
if diff<0:
    print("\nConjecture is wrong!")
else:
    print("\nConjecture not violated; difference is ", diff)

vbest:  [0.435874 0.435874 0.435874 0.435874 0.435874]
vconj:  [0.435874 0.435874 0.435874 0.435874 0.435874]

Estimated Imax value:  4.940510372207425
Conjectured Imax value:  4.940510372207472

Conjecture not violated; difference is  4.707345624410664e-14


In [9]:
#recap

print("w :",w)
print("vbest :",vbest.round(9))
print("vconj :",vconj.round(9))
print("lies within conjectured regime up to the Imax accuracy = ",diff)

w : [0.07633751 0.12426679 0.15111039 0.30446614 0.34381918]
vbest : [0.19007846 0.19362462 0.19927705 0.20562211 0.21139777]
vconj : [0.2 0.2 0.2 0.2 0.2]
lies within conjectured regime up to the Imax accuracy =  0.0013556497895006459


In [ ]:
print("Finished!")

In [9]:
vbest

array([0.43587378, 0.43587388, 0.4358739 , 0.43587391, 0.43587395])